# eNanoMapper API guide

## eNanoMapper database background

- FP7 project eNanoMapper http://www.enanomapper.net/
- eNanoMapper database implementation: AMBIT software http://ambit.sf.net
- publication https://www.beilstein-journals.org/bjnano/articles/6/165
- eNanoMapper prototype database https://data.enanomapper.net 

## Nanosafety data 

- NanoSafety data compiled in eNanoMapper databases: https://search.data.enanomapper.net/ 
- Each project data is imported into one enanoMapper instance, e.g. https://apps.ideaconsult.net/nanoreg1  
  - AMBIT REST API 
- Federated search view across multiple databases are available at https://search.data.enanomapper.net 
  - Solr REST API


## eNanoMapper data model

![data model](http://ambit.sourceforge.net/enanomapper/templates/images/data_model.png)
http://ambit.sourceforge.net/enanomapper/templates/convertor_how.html

## eNanoMapper database API

Swagger API docs at http://enanomapper.github.io/API/

In [1]:
from solrscope import aa
import ipywidgets as widgets
import requests
from importlib import reload 
from solrscope import client_solr
from solrscope import client_ambit
import pandas as pd
import numpy as np
import json

In [3]:
print('Select enanoMapper database:')
style = {'description_width': 'initial'}
config,config_servers, config_security = aa.parseOpenAPI3(config="enanomapper_public.yaml",auth=None)    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
display(service_widget)

Select enanoMapper database:


Dropdown(description='Service:', options=('https://apps.ideaconsult.net/nanoreg1', 'https://data.enanomapper.n…

### What is in the database ?

In [4]:
service_uri=service_widget.value

cli_facets = client_ambit.AMBITFacets(service_uri)
r = cli_facets.get(page=0,pagesize=1000)
if r.status_code==200:

    facets = cli_facets.parse(r.json())    
    print(json.dumps(facets, indent=4))   
else:
    facets = None
    print(r.status_code)

Sending request to https://apps.ideaconsult.net/nanoreg1/query/study params {'media': 'application/json', 'page': 0, 'pagesize': 1000}
[
    {
        "value": "6.1.5. Toxicity to aquatic algae and cyanobacteria",
        "endpoint": "EC_ALGAETOX_SECTION",
        "count": 20,
        "substancescount": -1,
        "uri": "https://apps.ideaconsult.net/nanoreg1/substance?type=endpointcategory&search=EC_ALGAETOX_SECTION",
        "subcategory": "ECOTOX",
        "subcategoryuri": "https://apps.ideaconsult.net/nanoreg1/substance?type=topcategory&search=ECOTOX",
        "bundles": {}
    },
    {
        "value": "6.1.3. Short-term toxicity to aquatic inverterbrates",
        "endpoint": "EC_DAPHNIATOX_SECTION",
        "count": 30,
        "substancescount": -1,
        "uri": "https://apps.ideaconsult.net/nanoreg1/substance?type=endpointcategory&search=EC_DAPHNIATOX_SECTION",
        "subcategory": "ECOTOX",
        "subcategoryuri": "https://apps.ideaconsult.net/nanoreg1/substance?type=

In [5]:
df=pd.DataFrame(facets)
display(df[["subcategory","endpoint","value","count"]])

In [6]:
#endpoints
cli_facets = client_ambit.AMBITFacets(service_uri,key="/experiment_endpoints")

r = cli_facets.get(page=0,pagesize=100,params={"top":"ECOTOX"})
if r.status_code==200:

    facets = cli_facets.parse(r.json())    
    #print(json.dumps(facets, indent=4))   
    df=pd.DataFrame(facets)
    display(df)
else:
    substances = None
    print(r.status_code)

Sending request to https://apps.ideaconsult.net/nanoreg1/query/experiment_endpoints params {'top': 'ECOTOX', 'media': 'application/json', 'page': 0, 'pagesize': 100}


###  Substance queries
#### All gold nanoparticles

In [7]:
reload(client_ambit)

service_uri=service_widget.value

cli_materials = client_ambit.AMBITSubstance(service_uri)
r = cli_materials.get(params={'search': 'NPO_401','type' : 'substancetype'},page=0,pagesize=10)
if r.status_code==200:

    substances = cli_materials.parse(r.json())    
    print(json.dumps(substances, indent=4))    
else:
    substances = None
    print(r.status_code)

Sending request to https://apps.ideaconsult.net/nanoreg1/substance params {'search': 'NPO_401', 'type': 'substancetype', 'media': 'application/json', 'page': 0, 'pagesize': 10}
[
    {
        "URI": "https://apps.ideaconsult.net/nanoreg1/substance/NNRG-9204c38f-ac08-e003-bd75-97239871d602",
        "ownerUUID": "NNRG-04022171-cf37-b07b-b933-a6b8d57883ed",
        "ownerName": "NANoREG",
        "i5uuid": "NNRG-9204c38f-ac08-e003-bd75-97239871d602",
        "name": "Au 13 nm",
        "publicname": "Au@PBPK",
        "format": "TNOEXP",
        "substanceType": "NPO_401",
        "referenceSubstance": {
            "i5uuid": null,
            "uri": "https://apps.ideaconsult.net/nanoreg1/query/compound/search/all?search=null"
        },
        "composition": [],
        "externalIdentifiers": [
            {
                "type": "Material code",
                "id": "Au@PBPK"
            },
            {
                "type": "NANoREG supplier",
                "id": "KI"
      

#### Retrieve physchem data for selected substances

In [8]:
for substance in substances:
    print(substance['URI'])    
    cli = client_ambit.AMBITSubstanceStudy(substance['URI'])
    r = cli.get(params={'category': 'PC_GRANULOMETRY_SECTION','top' : 'P-CHEM'},page=0,pagesize=2)
    #print(r.json())
    print(json.dumps(r.json(), indent=4))    

https://apps.ideaconsult.net/nanoreg1/substance/NNRG-9204c38f-ac08-e003-bd75-97239871d602
Sending request to https://apps.ideaconsult.net/nanoreg1/substance/NNRG-9204c38f-ac08-e003-bd75-97239871d602/study params {'category': 'PC_GRANULOMETRY_SECTION', 'top': 'P-CHEM', 'media': 'application/json', 'page': 0, 'pagesize': 2}
{
    "study": [
        {
            "uuid": "NRSZ-55e3f22e-f795-ae96-2ae5-a7c103503358",
            "investigation_uuid": null,
            "assay_uuid": "609caf44-b732-b054-dda5-860569e7e92e",
            "owner": {
                "substance": {
                    "uuid": "NNRG-9204c38f-ac08-e003-bd75-97239871d602"
                },
                "company": {
                    "uuid": "NNRG-04022171-cf37-b07b-b933-a6b8d57883ed",
                    "name": "NANoREG"
                }
            },
            "citation": {
                "title": "Provided",
                "year": "0",
                "owner": "KI"
            },
            "updated": 

#### Substance compositions

In [9]:
reload(client_ambit)
for substance in substances:
  
    print(substance['URI'])    
    cli = client_ambit.AMBITSubstanceComposition(substance['URI'])
    r = cli.get()
    compositions = cli.parse(r.json())
    for composition in compositions:
        print("-------------------------------------------------------------------------")
        print(composition['relation'])
        print(composition['proportion'])        
        print(composition['component']['compound']['cas'])
        print(composition['component']['compound']['name'])
        
        cli_cmp = client_ambit.AMBITCompound(root_uri=composition['component']['compound']['URI'],resource=None)
        response = cli_cmp.get(media="chemical/x-mdl-sdfile",pagesize=1)
        
        if response.status_code == 200:
            print(response.text) 
            

https://apps.ideaconsult.net/nanoreg1/substance/NNRG-9204c38f-ac08-e003-bd75-97239871d602
Sending request to https://apps.ideaconsult.net/nanoreg1/substance/NNRG-9204c38f-ac08-e003-bd75-97239871d602/composition params {'media': 'application/json', 'page': 0, 'pagesize': 10}
-------------------------------------------------------------------------
HAS_CORE
{'typical': {'precision': None, 'value': 0.0, 'unit': None}, 'real': {'lowerPrecision': None, 'lowerValue': 0.0, 'upperPrecision': None, 'upperValue': 0.0, 'unit': None}, 'function_as_additive': None}


Sending request to https://apps.ideaconsult.net/nanoreg1/compound/29726 params {'media': 'chemical/x-mdl-sdfile', 'page': 0, 'pagesize': 1}

  CDK     0331191851

  1  0  0  0  0  0  0  0  0  0999 V2000
    0.0000    0.0000    0.0000 Au  0  0  0  0  0  0  0  0  0  0  0  0

$$$$


#### Investigation
results in a tabular form

In [10]:
reload(client_ambit)
cli_investigation= client_ambit.AMBITInvestigation(service_uri)
r = cli_investigation.get(params={'search': 'PC_GRANULOMETRY_SECTION','type' : 'bystudytype'},page=0,pagesize=100)
if r.status_code==200:

    results = cli_investigation.parse(r.json())    
    print(json.dumps(results, indent=4))    
else:
    df=None
    print(r.status_code)

Sending request to https://apps.ideaconsult.net/nanoreg1/investigation params {'search': 'PC_GRANULOMETRY_SECTION', 'type': 'bystudytype', 'media': 'application/json', 'page': 0, 'pagesize': 100}
[
    {
        "name": "NM-402 (MWCNT 12.7 nm)",
        "publicname": "JRCNM04002a",
        "owner_name": "NANoREG",
        "topcategory": "P-CHEM",
        "endpointcategory": "PC_GRANULOMETRY_SECTION",
        "endpoint": "SIZE",
        "document_uuid": "NRDM-00000000-0000-0000-0000-000000000001",
        "guidance": "DLS",
        "reference": "final test live version",
        "reference_owner": "TNO",
        "idresult": 425931,
        "effectendpoint": "HYDRODYNAMIC DIAMETER",
        "unit": "nm",
        "loValue": 546.33,
        "errQualifier": "SD",
        "err": 0.13,
        "type_s": "study",
        "s_uuid": "NNRG-ea97c99b-e936-7dcf-b048-1ef314545e86",
        "substanceType": "NPO_354",
        "reference_year": 2016,
        "content": "JRC - IHCP",
        "iuuid": "3

In [11]:
df=pd.DataFrame(results)
display(df.head())

# Aggregated search

- Using Solr-powered free text and faceted search over several eNanoMapper database instances
- see https://search.data.enanomapper.net (web app) and  https://api.ideaconsult.net for API access


### Service selection

In [ ]:
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
display(service_widget)

In [ ]:
if config_security!='':
    print('Enter `{}` you have received upon subscription to http://api.ideaconsult.net'.format(config_security) )
    style = {'description_width': 'initial'}
    apikey_widget=widgets.Text(
        placeholder='',
        description=config_security,
        disabled=False,
        style=style
    )
    display(apikey_widget)

In [ ]:
service_uri=service_widget.value
print("Sending queries to {}".format(service_uri))


### Faceted search 
#### [Facets] Get all cell types

In [ ]:
reload(client_solr)

facets = client_solr.Facets()
query=facets.getQuery(query="*:*",facets=["E.cell_type_s"],fq="type_s:params")
#print(query)
r = client_solr.post(service_uri,query=query,auth=aa.GraviteeAuth(apikey_widget.value))
response_json=r.json()
if r.status_code==200:
    facets.parse(response_json['facets'])
else:
    print(r.status_code)


#### [Facets] Get all protocols per endpoint for titanium dioxide nanoparticles (NPO_1486)

In [ ]:
fields=["topcategory_s","endpointcategory_s","guidance_s"]
query=facets.getQuery(query="substanceType_s:NPO_1486",fq="type_s:study",facets=fields)
print(query)
r = client_solr.post(service_uri,query=query,auth=aa.GraviteeAuth(apikey_widget.value))
print(r.status_code)
if r.status_code==200:
    facets.parse(r.json()['facets'])
else:
    print(r.status_code)

#### [Facets] Get all methods

In [ ]:
fields=["topcategory_s","endpointcategory_s","E.method_s","E.sop_reference_s"]
query=facets.getQuery(query="*:*",fq="type_s:params",facets=fields)
print(query)
r = client_solr.post(service_uri,query=query,auth=aa.GraviteeAuth(apikey_widget.value))
print(r.status_code)
if r.status_code==200:
    facets.parse(r.json()['facets'])
else:
    print(r.status_code)

#### [Facets] Get all material types

In [ ]:
fields=["substanceType_hs","publicname_hs","name_hs","dbtag_hss"]
query=facets.getQuery(fq="type_s:substance",facets=fields)
#print(query)
r = client_solr.post(service_uri,query=query,auth=aa.GraviteeAuth(apikey_widget.value))
print(r.status_code)
if r.status_code==200:
    facets.parse(r.json()['facets'])
else:
    print(r.status_code)

#### [Facets]  Get all endpoints for nanotubes

In [ ]:
query=facets.getQuery(query="carbon nanotube",facets=["topcategory_s","endpointcategory_s","effectendpoint_s","unit_s"],fq="type_s:study")
#print(query)
r = client_solr.post(service_uri,query=query,auth=aa.GraviteeAuth(apikey_widget.value))
print(r.status_code)
#print(r.json()['facets'])
if r.status_code==200:
    facets.parse(r.json()['facets'])
else:
    print(r.status_code)

### Retrieve experimental data

#### Physchem example - MWCNT size

In [ ]:
reload(client_solr)
study = client_solr.StudyDocuments()
filter = {'topcategory_s':'P-CHEM', 'endpointcategory_s':'PC_GRANULOMETRY_SECTION' }
study.setStudyFilter(filter)
print(study.getSettings())
#all TiO2 NPO_1486
query = study.getQuery(textfilter='substanceType_s:NPO_354',rows=10000)
r = client_solr.post(service_uri,query=query,auth=aa.GraviteeAuth(apikey_widget.value))

In [ ]:
#parse the data
if r.status_code==200:
    study = client_solr.StudyDocuments()
    rows = study.parse(r.json()['response']['docs'])
    df = study.rows2frame(rows)
    rows=None
    uuids = ['uuid.investigation','uuid.assay','uuid.document','uuid.substance']
    df.sort_values(by=uuids)
    display(df.head(50))
else:
    print(r.status_code)

In [ ]:
#Group by material and endpoint
groups=[]

groups.append("m.public.name")
#groups.append("x.params.E.method")
#groups.append("p.guidance")
groups.append("x.params.MEDIUM")
groups.append("value.endpoint")
groups.append("value.endpoint_type")
groups.append("value.unit")
print(groups)

tmp=df.groupby(by=groups).agg({"value.range.lo" : ["mean","std","count"]}).reset_index()
(tmp)

#### Tox example - TiO2 cell viability

In [ ]:
reload(client_solr)
study = client_solr.StudyDocuments()
filter = {'topcategory_s':'TOX', 'endpointcategory_s':'ENM_0000068_SECTION' }
study.setStudyFilter(filter)
print(study.getSettings())
#all TiO2 NPO_1486
query = study.getQuery(textfilter='substanceType_s:NPO_354',rows=10000)
r = client_solr.post(service_uri,query=query,auth=aa.GraviteeAuth(apikey_widget.value))

In [ ]:
#parse the data
if r.status_code==200:
    study = client_solr.StudyDocuments()
    rows = study.parse(r.json()['response']['docs'])
    df = study.rows2frame(rows)
    rows=None
    uuids = ['uuid.investigation','uuid.assay','uuid.document','uuid.substance']
    df.sort_values(by=uuids)
    display(df.head(50))
else:
    print(r.status_code)

In [ ]:
groups=[]

groups.append("m.public.name")
groups.append("uuid.assay")
groups.append("uuid.document")
#groups.append("x.params.E.method")
#groups.append("p.guidance")
groups.append("x.params.MEDIUM")
groups.append("x.params.E.cell_type")
groups.append("x.conditions.material")
groups.append("value.endpoint")
groups.append("value.endpoint_type")
groups.append("value.unit")
print(groups)

tmp=df.groupby(by=groups).agg({"value.range.lo" : ["mean","std","count"]}).reset_index()
display(tmp)

.